# imports from Python 3

In [1]:
from __future__ import print_function

# CONSTANTS

In [2]:
# Database Access
JDBC_DRIVER_FILE_PATH =\
    '/Applications/PostgreSQL 9.4/postgresql-9.4-1205.jdbc42.jar'
DB_CONNECT =\
    'jdbc:postgresql://chicagoboothanalytics.clloyfrn47sj.us-west-1.rds.amazonaws.com:5432/ChicagoBoothAnalytics?user=BoothAnalytics&password=123BACmoderator'

# Database Table Names
APP_NAME =\
    'ChicagoBoothAnalytics_app'
DB_TABLE_NAME___BUSINESSSECTOR =\
    '"%s_businesssector"' % APP_NAME
DB_TABLE_NAME___GEOGREGION =\
    '"%s_geogregion"' % APP_NAME
DB_TABLE_NAME___ORG =\
    '"%s_org"' % APP_NAME
DB_TABLE_NAME___ORG_BUSINESSSECTOR_M2M =\
    '"%s_org_business_sectors"' % APP_NAME
DB_TABLE_NAME___ORG_GEOGREGION_M2M =\
    '"%s_org_geog_regions"' % APP_NAME
DB_TABLE_NAME___PERSON =\
    '"%s_person"' % APP_NAME
DB_TABLE_NAME___ROLELVEL =\
    '"%s_rolelevel"' % APP_NAME
DB_TABLE_NAME___ROLE =\
    '"%s_role"' % APP_NAME
DB_TABLE_NAME___CAREEROPPORTUNITY_GEOGREGION_M2M =\
    '"%s_careeropportunity_geog_regions"' % APP_NAME
DB_TABLE_NAME___CAREEROPPORTUNITY =\
    '"%s_careeropportunity"' % APP_NAME
DB_TABLE_NAME___FACTTYPE =\
    '"%s_facttype"' % APP_NAME
DB_TABLE_NAME___ORGFACT =\
    '"%s_orgfact"' % APP_NAME
    
NB_ROWS_TO_SHOW = 9

# import Spark; initiate SparkContext & HiveContext

In [3]:
if 'pyspark' not in vars():
    from findspark import init
    init()
    import pyspark
    sc = pyspark.SparkContext()
    hc = pyspark.sql.HiveContext(sc)
    
print('SparkContext: %s' % str(sc))
print('HiveContext: %s' % str(hc))

SparkContext: <pyspark.context.SparkContext object at 0x106d49c90>
HiveContext: <pyspark.sql.context.HiveContext object at 0x10848f8d0>


# ORIGINAL DATABASE TABLES

## "BusinessSector" table

In [4]:
BusinessSector_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___BUSINESSSECTOR)\
    .sort('name')

BusinessSector_DDF.registerTempTable('BusinessSector')

BusinessSector_DDF.cache()

BusinessSector_DDF.toPandas()

,id,name
0,28,Advertising
1,21,Automotive
2,20,Beverages
3,1,Consulting
4,16,Consumer Goods
5,25,Creative
6,7,E-Commerce
7,29,Education
8,31,Entertainment
9,13,Financial Services


## "GeogRegion" table

In [5]:
GeogRegion_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___GEOGREGION)\
    .sort('name')

GeogRegion_DDF.registerTempTable('GeogRegion')

GeogRegion_DDF.cache()

GeogRegion_DDF.toPandas()

,id,name
0,8,"Atlanta, GA"
1,13,"Austin, TX"
2,14,"Boston (Greater), MA"
3,6,"Boston, MA"
4,9,"Chicago (Greater), IL"
5,2,"Chicago, IL"
6,11,"Cincinnati, OH"
7,12,"Los Angeles (Greater), LA"
8,10,"Los Angeles, CA"
9,5,"New York City, NY"


## "Org" table

In [6]:
Org_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORG)\
    .sort('name')

Org_DDF.registerTempTable('Org')

Org_DDF.cache()

Org_DDF.show(NB_ROWS_TO_SHOW)

+---+---------+
| id|     name|
+---+---------+
| 83|       3M|
| 84|   Abbott|
| 46|Accenture|
| 44|   Acxiom|
|  8| AdYapper|
|  4|   Adatao|
|  7|    Adobe|
|  9|   Affirm|
| 53|   AirBnB|
+---+---------+
only showing top 9 rows



## "Org_BusinessSector" table

In [7]:
Org_BusinessSector_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORG_BUSINESSSECTOR_M2M)

Org_BusinessSector_m2m_DDF.registerTempTable('Org_BusinessSector_m2m')

Org_BusinessSector_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------------+
| id|org_id|businesssector_id|
+---+------+-----------------+
|  1|     3|                4|
|  2|     3|                7|
|  4|     4|                4|
|  6|     5|                4|
|  8|     6|                4|
|  9|     7|                4|
| 10|     8|                4|
| 11|     8|                5|
| 12|     9|                9|
+---+------+-----------------+
only showing top 9 rows



In [8]:
Org_BusinessSector_DDF = hc.sql(
    'SELECT \
        Org_BusinessSector_m2m.id as id, \
        Org_BusinessSector_m2m.org_id as org_id, \
        Org_BusinessSector_m2m.businesssector_id as businesssector_id, \
        Org.name as org, \
        BusinessSector.name as business_sector \
    FROM \
        Org_BusinessSector_m2m \
        LEFT JOIN Org ON Org_BusinessSector_m2m.org_id = Org.id \
        LEFT JOIN BusinessSector on Org_BusinessSector_m2m.businesssector_id = BusinessSector.id')\
    .sort('org', 'business_sector')

Org_BusinessSector_DDF.registerTempTable('Org_BusinessSector')

Org_BusinessSector_DDF.cache()

Org_BusinessSector_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------------+---------+---------------+
| id|org_id|businesssector_id|      org|business_sector|
+---+------+-----------------+---------+---------------+
|110|    83|               25|       3M|       Creative|
|111|    84|               26|   Abbott|     Healthcare|
|112|    84|               27|   Abbott| Pharmaceutical|
| 68|    46|                1|Accenture|     Consulting|
|113|    44|               28|   Acxiom|    Advertising|
|114|    44|                5|   Acxiom|      Marketing|
| 11|     8|                5| AdYapper|      Marketing|
| 10|     8|                4| AdYapper|     Technology|
|  4|     4|                4|   Adatao|     Technology|
+---+------+-----------------+---------+---------------+
only showing top 9 rows



## "Org_GeogRegion" table

In [9]:
Org_GeogRegion_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORG_GEOGREGION_M2M)

Org_GeogRegion_m2m_DDF.registerTempTable('Org_GeogRegion_m2m')

Org_GeogRegion_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-------------+
| id|org_id|geogregion_id|
+---+------+-------------+
|  1|     3|            1|
|  2|     3|            4|
|  3|     4|            4|
|  4|     5|            4|
|  5|     6|            3|
|  6|     7|            4|
|  7|     7|            7|
|  8|     9|            3|
|  9|    10|            2|
+---+------+-------------+
only showing top 9 rows



In [10]:
Org_GeogRegion_DDF = hc.sql(
    'SELECT \
        Org_GeogRegion_m2m.id as id, \
        Org_GeogRegion_m2m.org_id as org_id, \
        Org_GeogRegion_m2m.geogregion_id as geogregion_id, \
        Org.name as org, \
        GeogRegion.name as geog_region \
    FROM \
        Org_GeogRegion_m2m \
        LEFT JOIN Org ON Org_GeogRegion_m2m.org_id = Org.id \
        LEFT JOIN GeogRegion ON Org_GeogRegion_m2m.geogregion_id = GeogRegion.id')\
    .sort('org', 'geog_region')

Org_GeogRegion_DDF.registerTempTable('Org_GeogRegion')

Org_GeogRegion_DDF.cache()

Org_GeogRegion_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-------------+--------+--------------------+
| id|org_id|geogregion_id|     org|         geog_region|
+---+------+-------------+--------+--------------------+
| 70|    44|            9|  Acxiom|Chicago (Greater)...|
|  3|     4|            4|  Adatao|San Francisco Bay...|
|  7|     7|            7|   Adobe|  Salt Lake City, UT|
|  6|     7|            4|   Adobe|San Francisco Bay...|
|  8|     9|            3|  Affirm|   San Francisco, CA|
| 53|    53|            3|  AirBnB|   San Francisco, CA|
|  9|    10|            2|Allstate|         Chicago, IL|
| 38|    38|            4|Alphabet|San Francisco Bay...|
|  2|     3|            4|  Amazon|San Francisco Bay...|
+---+------+-------------+--------+--------------------+
only showing top 9 rows



## "Person" table

In [11]:
Person_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___PERSON)\
    .selectExpr(
        'id',
        'CONCAT( \
            COALESCE( \
                IF(first_name_alias = "", NULL, first_name_alias), \
                first_name), \
            " ", \
            UPPER(last_name)) \
            as name',
        'last_name',
        'first_name',
        'first_name_alias',
        'IF(gender, "male", "female") as gender')\
    .sort('last_name')

Person_DDF.registerTempTable('Person')

Person_DDF.cache()

Person_DDF.show(NB_ROWS_TO_SHOW)

+---+--------------+---------+--------------+----------------+------+
| id|          name|last_name|    first_name|first_name_alias|gender|
+---+--------------+---------+--------------+----------------+------+
| 21|  Paul ARONSON|  Aronson|          Paul|                |  male|
|  2|   Sahil BAHRI|    Bahri|         Sahil|                |  male|
| 18|  Anju BHALDAR|  Bhaldar|          Anju|                |female|
| 16|Rudi BONAPARTE|Bonaparte|          Rudi|                |  male|
|  8|    Cheng CHEN|     Chen|         Cheng|                |  male|
| 11| John CIASULLI| Ciasulli|    John David|            John|  male|
|  7|Amanda COLLINS|  Collins|Amanda Marlene|          Amanda|female|
| 12|       Tony DO|       Do|         Thong|            Tony|  male|
| 23|Andrew EDELMAN|  Edelman| Andrew Cooper|          Andrew|  male|
+---+--------------+---------+--------------+----------------+------+
only showing top 9 rows



# "RoleLevel" table

In [12]:
RoleLevel_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ROLELVEL)\
    .sort('level_number_from_high_to_low')

RoleLevel_DDF.registerTempTable('RoleLevel')
    
RoleLevel_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------------------------+
| id| level|level_number_from_high_to_low|
+---+------+-----------------------------+
|  2| Board|                            1|
|  1|     C|                            2|
|  3|senior|                            3|
|  4|   mid|                            4|
|  5|junior|                            5|
+---+------+-----------------------------+



## "Role" table

In [13]:
Role_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ROLE)

Role_DDF.registerTempTable('Role')

Role_DDF = hc.sql(
    'SELECT \
        Role.id as id, \
        CONCAT(Role.title, " [", RoleLevel.level, "-level]") as title, \
        RoleLevel.level_number_from_high_to_low as level \
    FROM \
        Role \
        JOIN RoleLevel ON Role.level_id = RoleLevel.id')\
    .sort('level_number_from_high_to_low', 'title')

Role_DDF.registerTempTable('Role')

Role_DDF.cache()
    
Role_DDF.toPandas()

,id,title,level
0,2,Chair of the Board [Board-level],1
1,5,Director of the Board [Board-level],1
2,8,(Co-)Founder [C-level],2
3,19,Chief Analytics Officer (CAO) [C-level],2
4,1,Chief Executive Officer (CEO) [C-level],2
5,20,Chief Financial Officer (CFO) [C-level],2
6,21,Chief Operating Officer (COO) [C-level],2
7,18,Chief Technology Officer (CTO) [C-level],2
8,27,Director of Analytics [senior-level],3
9,3,Director of Data Science [senior-level],3


## "CareerOpportunity_GeogRegion" table

In [14]:
CareerOpportunity_GeogRegion_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___CAREEROPPORTUNITY_GEOGREGION_M2M)

CareerOpportunity_GeogRegion_m2m_DDF.registerTempTable('CareerOpportunity_GeogRegion_m2m')

CareerOpportunity_GeogRegion_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+--------------------+-------------+
| id|careeropportunity_id|geogregion_id|
+---+--------------------+-------------+
|  1|                   5|            2|
|  2|                   3|            3|
|  3|                   1|            3|
|  4|                   2|            4|
|  5|                   4|            3|
|  6|                   6|            1|
|  7|                   6|            4|
|  9|                   7|            4|
| 12|                   9|            4|
+---+--------------------+-------------+
only showing top 9 rows



In [15]:
CareerOpportunity_GeogRegion_DDF = hc.sql(
    'SELECT \
        CareerOpportunity_GeogRegion_m2m.careeropportunity_id as careeropportunity_id, \
        GeogRegion.name as geog_region \
    FROM \
        CareerOpportunity_GeogRegion_m2m \
        LEFT JOIN GeogRegion ON CareerOpportunity_GeogRegion_m2m.geogregion_id = GeogRegion.id')

CareerOpportunity_GeogRegion_DDF.registerTempTable('CareerOpportunity_GeogRegion')

CareerOpportunity_GeogRegion_DDF.cache()

CareerOpportunity_GeogRegion_DDF.show(NB_ROWS_TO_SHOW)

+--------------------+--------------------+
|careeropportunity_id|         geog_region|
+--------------------+--------------------+
|                   5|         Chicago, IL|
|                   3|   San Francisco, CA|
|                   1|   San Francisco, CA|
|                   2|San Francisco Bay...|
|                   4|   San Francisco, CA|
|                   6|         Seattle, WA|
|                   6|San Francisco Bay...|
|                   7|San Francisco Bay...|
|                   9|San Francisco Bay...|
+--------------------+--------------------+
only showing top 9 rows



In [16]:
CareerOpportunity_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___CAREEROPPORTUNITY)

CareerOpportunity_DDF.registerTempTable('CareerOpportunity')

CareerOpportunity_DDF = hc.sql(
    'SELECT \
        CareerOpportunity.id as id, \
        Org.id as org_id, \
        Org.name as org, \
        Role.title as role, \
        Role.level as role_level, \
        CareerOpportunity_GeogRegion.geog_region as geog_region, \
        CareerOpportunity.open as open, \
        CareerOpportunity.posting_date as posting_date, \
        CareerOpportunity.url as url \
    FROM \
        CareerOpportunity \
        LEFT JOIN Org ON CareerOpportunity.org_id = Org.id \
        LEFT JOIN Role ON CareerOpportunity.role_id = Role.id \
        LEFT JOIN CareerOpportunity_GeogRegion ON \
            CareerOpportunity.id = CareerOpportunity_GeogRegion.careeropportunity_id')\
    .sort(
        'org',
        'role_level',
        'role',
        'geog_region')\
    .sort('open', ascending=False)

CareerOpportunity_DDF.registerTempTable('CareerOpportunity')

CareerOpportunity_DDF.cache()

CareerOpportunity_DDF.show(NB_ROWS_TO_SHOW)

+---+------+--------------------+--------------------+----------+--------------------+----+------------+--------------------+
| id|org_id|                 org|                role|role_level|         geog_region|open|posting_date|                 url|
+---+------+--------------------+--------------------+----------+--------------------+----+------------+--------------------+
|  8|     4|              Adatao|Data Scientist [m...|         4|San Francisco Bay...|true|  2015-10-12|http://www.linked...|
| 10|     7|               Adobe|Data Scientist [m...|         4|San Francisco Bay...|true|  2015-10-06|http://www.linked...|
|  6|     1|Amazon Web Servic...|Machine Learning ...|         4|San Francisco Bay...|true|        null|                    |
|  6|     1|Amazon Web Servic...|Machine Learning ...|         4|         Seattle, WA|true|        null|                    |
|  2|    41|               Banjo|Senior Data Scien...|         4|San Francisco Bay...|true|  2015-09-19|https://www.ka

## "FactType" table

In [17]:
FactType_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___FACTTYPE)\
    .sort('label')
    
FactType_DDF.registerTempTable('FactType')

FactType_DDF.toPandas()

,id,label
0,3,LinkedIn profile URL
1,6,business school
2,5,business/work phone number
3,8,careers website
4,1,email address
5,2,hobby
6,4,website
7,7,year founded


## "OrgFact" table

In [18]:
OrgFact_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORGFACT)

OrgFact_DDF.registerTempTable('OrgFact')

OrgFact_DDF = hc.sql(
    'SELECT \
        OrgFact.id as id, \
        OrgFact.org_id as org_id, \
        OrgFact.fact_type_id as facttype_id, \
        Org.name as org, \
        FactType.label as fact_type, \
        OrgFact.fact as fact \
    FROM \
        OrgFact \
        LEFT JOIN Org ON OrgFact.org_id = Org.id \
        LEFT JOIN FactType ON OrgFact.fact_type_id = FactType.id')\
    .sort(
        'org',
        'fact_type',
        'fact')

OrgFact_DDF.registerTempTable('OrgFact')

OrgFact_DDF.cache()

OrgFact_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------+---------+------------+-------------+
| id|org_id|facttype_id|      org|   fact_type|         fact|
+---+------+-----------+---------+------------+-------------+
| 85|    83|          4|       3M|     website|       3M.com|
| 86|    84|          4|   Abbott|     website|   Abbott.com|
| 47|    46|          4|Accenture|     website|Accenture.com|
| 45|    44|          4|   Acxiom|     website|   Acxiom.com|
| 14|     8|          4| AdYapper|     website| AdYapper.com|
|  1|     4|          4|   Adatao|     website|   Adatao.com|
|  4|     4|          7|   Adatao|year founded|         2012|
| 13|     7|          4|    Adobe|     website|    Adobe.com|
| 39|     9|          4|   Affirm|     website|   Affirm.com|
+---+------+-----------+---------+------------+-------------+
only showing top 9 rows



# DERIVED TABLES

## "OrgOverview" table

In [19]:
# FactType_website table
FactType_website_DDF = hc.sql(
    'SELECT \
        id, \
        label \
    FROM \
        FactType \
    WHERE \
        label = "website"')


# OrgWebsites table
OrgFact_website_DDF =\
    OrgFact_DDF.join(
        FactType_website_DDF,
        on=OrgFact_DDF.facttype_id == FactType_website_DDF.id)\
    .select(
        'org_id',
        'fact')
    
OrgFact_website_DDF.registerTempTable('OrgFact_website')


# OrgNbOpenCareerOpportunities
OrgNbOpenCareerOpportunities_DDF = hc.sql(
    'SELECT \
        org_id, \
        COUNT(*) as nb_open_career_opportunities \
    FROM \
        CareerOpportunity \
    WHERE \
        open = true \
    GROUP BY \
       org_id')

OrgNbOpenCareerOpportunities_DDF.registerTempTable('OrgNbOpenCareerOpportunities')


# OrgOverview table
OrgOverview_DDF = hc.sql(
    'SELECT \
        Org.id as id, \
        Org.name as name, \
        COLLECT_SET(OrgFact_website.fact) as websites, \
        COLLECT_SET(Org_BusinessSector.business_sector) as business_sectors, \
        COLLECT_SET(Org_GeogRegion.geog_region) as geog_regions, \
        OrgNbOpenCareerOpportunities.nb_open_career_opportunities as nb_open_career_opportunities \
    FROM \
        Org \
        LEFT JOIN OrgFact_website ON Org.id = OrgFact_website.org_id \
        LEFT JOIN Org_BusinessSector ON Org.id = Org_BusinessSector.org_id \
        LEFT JOIN Org_GeogRegion ON Org.id = Org_GeogRegion.org_id \
        LEFT JOIN OrgNbOpenCareerOpportunities ON Org.id = OrgNbOpenCareerOpportunities.org_id \
    GROUP BY \
        Org.id, \
        Org.name, \
        OrgNbOpenCareerOpportunities.nb_open_career_opportunities')\
    .sort('name')

OrgOverview_DDF.toPandas()

,id,name,websites,business_sectors,geog_regions,nb_open_career_opportunities
0,83,3M,[3M.com],[Creative],[],NaN
1,84,Abbott,[Abbott.com],"[Healthcare, Pharmaceutical]",[],NaN
2,46,Accenture,[Accenture.com],[Consulting],[],NaN
3,44,Acxiom,[Acxiom.com],"[Advertising, Marketing]","[Chicago (Greater), IL]",NaN
4,8,AdYapper,[AdYapper.com],"[Marketing, Technology]",[],NaN
5,4,Adatao,[Adatao.com],[Technology],"[San Francisco Bay Area, CA]",1
6,7,Adobe,[Adobe.com],[Technology],"[Salt Lake City, UT, San Francisco Bay Area, CA]",1
7,9,Affirm,[Affirm.com],"[Financial Services: Consumer Finance, Technol...","[San Francisco, CA]",NaN
8,53,AirBnB,[AirBnB.com],"[Hospitality, Technology]","[San Francisco, CA]",NaN
9,85,Alcatel-Lucent,[Alcatel-Lucent.com],[Technology],[],NaN


## "CareerOpportunityList" table

In [20]:
CareerOpportunityList_DDF = hc.sql(
    'SELECT \
        id, \
        org, \
        role, \
        role_level, \
        COLLECT_SET(geog_region) as geog_regions, \
        posting_date, \
        url \
    FROM \
        CareerOpportunity \
    GROUP BY \
        id, \
        org, \
        role, \
        role_level, \
        posting_date, \
        url')\
    .sort(
        'org',
        'role_level',
        'role')\
    .sort('open', ascending=False)
    
CareerOpportunityList_DDF.toPandas()

,id,org,role,role_level,geog_regions,posting_date,url
0,8,Adatao,Data Scientist [mid-level],4,"[San Francisco Bay Area, CA]",2015-10-12,http://www.linkedin.com/jobs2/view/62346316
1,10,Adobe,Data Scientist [mid-level],4,"[San Francisco Bay Area, CA]",2015-10-06,http://www.linkedin.com/jobs2/view/76647155
2,6,Amazon Web Services (AWS),Machine Learning Scientist [mid-level],4,"[San Francisco Bay Area, CA, Seattle, WA]",None,
3,2,Banjo,Senior Data Scientist [mid-level],4,"[San Francisco Bay Area, CA]",2015-09-19,https://www.kaggle.com/jobs/16575/banjo-senior...
4,13,Facebook,Data Scientist [mid-level],4,"[San Francisco Bay Area, CA]",2015-10-13,http://www.linkedin.com/jobs2/view/77971276
5,14,G/O Digital,Associate Data Scientist [mid-level],4,"[Chicago, IL]",2015-10-08,
6,15,General Motors (GM),Data Analyst [mid-level],4,"[Austin, TX]",2015-09-29,
7,17,GitHub,Data Scientist [mid-level],4,"[San Francisco Bay Area, CA]",2015-10-28,http://linkedin.com/jobs2/view/81836493
8,9,GoDaddy,Data Scientist [mid-level],4,"[San Francisco Bay Area, CA]",2015-10-08,http://www.linkedin.com/jobs2/view/77910845
9,20,InstaCart,Data Scientist [mid-level],4,"[San Francisco Bay Area, CA]",2015-10-30,https://www.linkedin.com/jobs2/view/81868859
